### Generate Insight into the correlation between Error- and UQ Maps of the reconstruction (R1 vs R3 or R6)


In [1]:
import sqlite3
import pandas as pd
from pathlib import Path
import seaborn as sns
import matplotlib.pyplot as plt

# === Configurable Parameters ===
db_fpath  = Path('/home1/p290820/repos/Uncertainty-Quantification-Prostate-MRI/databases/master_habrok_20231106_v2.db')
table     = "uq_vs_error_correlation_std"  # or remove _debug if running final
verbose   = True

In [2]:
# === SQLite DB Utilities ===

def connect_db(db_path: Path) -> sqlite3.Connection:
    """Establish and return SQLite connection."""
    return sqlite3.connect(str(db_path))

def run_query(conn: sqlite3.Connection, query: str) -> pd.DataFrame:
    """Execute a SQL query and return the result as a DataFrame."""
    return pd.read_sql_query(query, conn)

def preview_table(conn: sqlite3.Connection, table_name: str, n: int = 5):
    """Print the first n rows of the specified table."""
    df = run_query(conn, f"SELECT * FROM {table_name} LIMIT {n};")
    display(df)

# === Establish Connection ===
conn = connect_db(db_fpath)

# === Preview Table (Optional) ===
if verbose:
    print(f"🔍 Preview of '{table}' table:")
    preview_table(conn, table)

🔍 Preview of 'uq_vs_error_correlation_std' table:


,pat_id,slice_idx,acc_factor,uq_method,region,mean_abs,mean_uq,std_uq,pearson_corr,spearman_corr
0,0003_ANON5046358,0,3,gaussian,slice,8.835545,0.000991,0.000257,0.367489,0.374207
1,0003_ANON5046358,1,3,gaussian,slice,9.663578,0.001003,0.000257,0.413001,0.388464
2,0003_ANON5046358,2,3,gaussian,slice,8.432629,0.001011,0.000259,0.392491,0.368357
3,0003_ANON5046358,3,3,gaussian,slice,8.865566,0.001011,0.000253,0.339506,0.329187
4,0003_ANON5046358,4,3,gaussian,slice,9.475834,0.001019,0.000259,0.414462,0.417164


1. Which UQ Method Aligns Better With Error?
🔍 Compare Spearman/Pearson correlation between UQ and error per method

✅ Next: Boxplot Per Slice × Region
Let’s expand your boxplot to include region, so we don’t over-average.

In [3]:
# --- Pull all slice-level correlations including region ---
query = f"""
SELECT uq_method, acc_factor, region, spearman_corr
FROM {table}
WHERE spearman_corr IS NOT NULL;
"""
df = run_query(conn, query)

# --- Faceted Violin Plots by Acceleration Factor with Grid Lines ---
g = sns.catplot(
    data=df,
    x="region", y="spearman_corr", hue="uq_method",
    col="acc_factor", kind="violin", split=True,
    palette="Set2", height=5, aspect=1
)

# Add grid lines to each subplot
for ax in g.axes.flat:
    ax.grid(True)

g.fig.subplots_adjust(top=0.85)
g.fig.suptitle("Spearman Correlation by Region, Method, and Acceleration", fontsize=14)
plt.show()


🧪 Next Cell: Mixed Effects Model (in Python via statsmodels)

In [4]:
import statsmodels.formula.api as smf

# Filter to valid rows
df_model = df.dropna(subset=["spearman_corr"])

# Fit a linear mixed effects model with patient as random effect
model = smf.mixedlm(
    "spearman_corr ~ uq_method * acc_factor * region",
    df_model,
    groups=df_model["pat_id"]
).fit()

# Display summary
print(model.summary())

df["region"] = df["region"].astype("category")
df["uq_method"] = df["uq_method"].astype("category")
df["pat_id"] = df["pat_id"].astype("category")

KeyError: 'pat_id'